In [1]:
!nvidia-smi

Thu Oct 15 02:24:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    39W / 300W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
!pip install -q transformers

In [2]:
!wget -q https://raw.githubusercontent.com/tensorflow/examples/master/tensorflow_examples/lite/model_maker/core/task/hub_loader.py

In [2]:
from hub_loader import HubKerasLayerV1V2

import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import transformers
import matplotlib.pyplot as plt

%matplotlib inline

# Fix random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF Version:  2.4.0-dev20200810
Eager mode:  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [3]:
data_dir = tf.keras.utils.get_file(
      fname='SST-2.zip',
      origin='https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8',
      extract=True)
data_dir = os.path.join(os.path.dirname(data_dir), 'SST-2')

In [4]:
train = os.path.join(data_dir, "train.tsv")
valid = os.path.join(data_dir, "dev.tsv")
test = os.path.join(data_dir, "test.tsv")

train_dataset = pd.read_csv(train, sep='\t')
valid_dataset = pd.read_csv(valid, sep='\t')
test_dataset = pd.read_csv(test, sep='\t')

In [5]:
train_dataset.head()

,sentence,label
0,hide new secretions from the parental units,0
1,"contains no wit , only labored gags",0
2,that loves its characters and communicates som...,1
3,remains utterly satisfied to remain the same t...,0
4,on the worst revenge-of-the-nerds clichés the ...,0


In [6]:
train_reviews = train_dataset['sentence'].values
train_sentiments = train_dataset['label'].values

valid_reviews = valid_dataset['sentence'].values
valid_sentiments = valid_dataset['label'].values

test_reviews = test_dataset['sentence'].values

train_reviews.shape, valid_reviews.shape, test_reviews.shape

((67349,), (872,), (1821,))

In [7]:
tokenizer = transformers.MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')

In [8]:
# Referred from here - https://github.com/dipanjanS/deep_transfer_learning_nlp_dhs2019/blob/master/notebooks/5%20-%20Transformers%20-%20BERT.ipynb
import tqdm

def create_bert_input_features(tokenizer, docs, max_seq_length):
    
    all_ids, all_masks, all_segments= [], [], []
    for doc in tqdm.tqdm(docs, desc="Converting docs to features"):
        tokens = tokenizer.tokenize(doc)
        if len(tokens) > max_seq_length-2:
            tokens = tokens[0 : (max_seq_length-2)]
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        ids = tokenizer.convert_tokens_to_ids(tokens)
        masks = [1] * len(ids)
        # Zero-pad up to the sequence length.
        while len(ids) < max_seq_length:
            ids.append(0)
            masks.append(0)
        segments = [0] * max_seq_length
        all_ids.append(ids)
        all_masks.append(masks)
        all_segments.append(segments)
    encoded = np.array([all_ids, all_masks, all_segments])
    return encoded

In [9]:
MAX_SEQ_LENGTH = 128

train_features_ids, train_features_masks, train_features_segments = create_bert_input_features(tokenizer, 
                                                                                               train_reviews, 
                                                                                               max_seq_length=MAX_SEQ_LENGTH)
val_features_ids, val_features_masks, val_features_segments = create_bert_input_features(tokenizer, 
                                                                                         valid_reviews, 
                                                                                         max_seq_length=MAX_SEQ_LENGTH)

print('Train Features:', train_features_ids.shape, train_features_masks.shape, train_features_segments.shape)
print('Val Features:', val_features_ids.shape, val_features_masks.shape, val_features_segments.shape)

Converting docs to features: 100%|██████████| 872/872 [00:00<00:00, 3253.03it/s]

Train Features: (67349, 128) (67349, 128) (67349, 128)
Val Features: (872, 128) (872, 128) (872, 128)


In [10]:
# Create TensorFlow datasets for better performance
train_ds = (
    tf.data.Dataset
    .from_tensor_slices(((train_features_ids, train_features_masks, train_features_segments), train_sentiments))
    .shuffle(2048)
    .batch(24)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
    
valid_ds = (
    tf.data.Dataset
    .from_tensor_slices(((val_features_ids, val_features_masks, val_features_segments), valid_sentiments))
    .batch(24)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [11]:
# Reference - https://github.com/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/core/task/model_spec.py

input_word_ids = tf.keras.layers.Input(
      shape=(MAX_SEQ_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_mask = tf.keras.layers.Input(
    shape=(MAX_SEQ_LENGTH,), dtype=tf.int32, name='input_mask')
input_type_ids = tf.keras.layers.Input(
    shape=(MAX_SEQ_LENGTH,), dtype=tf.int32, name='input_type_ids')
  
bert_model = HubKerasLayerV1V2(
    'https://tfhub.dev/google/mobilebert/uncased_L-24_H-128_B-512_A-4_F-4_OPT/1',
    signature='tokens',
    output_key='pooled_output',
    trainable=True)

pooled_output = bert_model({
    'input_ids': input_word_ids,
    'input_mask': input_mask,
    'segment_ids': input_type_ids
})

dense1 = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(256, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)
output = tf.keras.layers.Dense(1, activation='sigmoid')(drop2)

model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], 
                       outputs=output)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=3e-5), 
              loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/mobilebert/uncased_L-24_H-128_B-512_A-4_F-4_OPT/1'.
INFO:absl:Downloaded https://tfhub.dev/google/mobilebert/uncased_L-24_H-128_B-512_A-4_F-4_OPT/1, Total size: 103.86MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/mobilebert/uncased_L-24_H-128_B-512_A-4_F-4_OPT/1'.


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
hub_keras_layer_v1v2 (HubKerasL (None, 512)          24581888    input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [12]:
start = time.time()
model.fit(train_ds,
    validation_data=valid_ds,
    epochs=3)
end = time.time()
print("Training took {} seconds.".format(end - start))

Epoch 1/3
2807/2807 [==============================] - 359s 128ms/step - loss: 0.5032 - accuracy: 0.7654 - val_loss: 0.2402 - val_accuracy: 0.9048
Epoch 2/3
2807/2807 [==============================] - 306s 109ms/step - loss: 0.1580 - accuracy: 0.9423 - val_loss: 0.2441 - val_accuracy: 0.9083
Epoch 3/3
2807/2807 [==============================] - 305s 109ms/step - loss: 0.1010 - accuracy: 0.9656 - val_loss: 0.2861 - val_accuracy: 0.9025
Training took 971.4864466190338 seconds.


In [13]:
!mkdir mobilebert
model.save("mobilebert")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: mobilebert/assets


INFO:tensorflow:Assets written to: mobilebert/assets


In [14]:
converter = tf.lite.TFLiteConverter.from_saved_model("mobilebert")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("mobilebert_2_sst_seq_128.tflite", "wb").write(tflite_model)

INFO:absl:Using new converter: If you encounter a problem please file a bug. You can opt-out by setting experimental_new_converter=False


26698576

In [15]:
!ls -lh mobilebert_2_sst_seq_128.tflite

-rw-r--r-- 1 jupyter jupyter 26M Oct 15 02:42 mobilebert_2_sst_seq_128.tflite
